# Solutions to Project Euler #1-10

## Problem 1
[Multiples of 3 and 5](http://projecteuler.net/problem=1)
> If we list all the natural numbers below 10 that are multiples of 3 or 5, we get 3, 5, 6 and 9. The sum of these multiples is 23. Find the sum of all the multiples of 3 or 5 below 1000.


This is a typical foobar problem. I take union of two sets---multiples of 3 and multiples of 5.

In [5]:
def prob001(a:int=3, b:int=5, below:int=1000) -> int:
    """
    >>> prob001(3, 5, 10)
    22
    """
    set1 = set(range(a, below, a))
    set2 = set(range(b, below, b))
    return sum(set1 | set2)

In [6]:
%doctest_mode

Exception reporting mode: Plain
Doctest mode is: ON


In [7]:
prob001()

233168

In [8]:
%timeit prob001()

10000 loops, best of 3: 50.2 µs per loop


**Another Solution:** filtering might be nice.

In [9]:
def prob001b(a:int=3, b:int=5, below:int=1000) -> int:
    return sum(i for i in range(below) if i % a == 0 or i % b == 0)

In [10]:
prob001b()

233168

In [11]:
%timeit prob001b()

1000 loops, best of 3: 348 µs per loop


## Problem 2
[Even Fibonacci numbers](http://projecteuler.net/problem=2)
> Each new term in the Fibonacci sequence is generated by adding the previous two terms. By starting with 1 and 2, the first 10 terms will be: 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, ... By considering the terms in the Fibonacci sequence whose values do not exceed four million, find the sum of the even-valued terms.

Fibonacci sequence may be generated from repeated map of two-integer state. Or, in matrix form,

$$
\begin{align*}
    a_{n+1} &= b_n \\
    b_{n+1} &= a_n + b_n 
\end{align*}
$$

with $a_0 = 0$ and  $b_0 = 1$.

Also note that an infinite series is nicely handled with the Python generator.

In [2]:
import itertools as it

def prob002(maxval:int=4000000) -> int:
    def fibs():
        i, j = 0, 1
        while True:
            (i, j) = (j, i + j)
            yield j 
    
    fibseq = fibs()
    return sum(n for n in it.takewhile(lambda x: x < maxval, fibseq) if n % 2 == 0)

In [3]:
prob002()

4613732

In [4]:
%timeit prob002()

10000 loops, best of 3: 20 µs per loop


## Problem 3
[Largest prime factor](http://projecteuler.net/problem=3)
> The prime factors of 13195 are 5, 7, 13 and 29. What is the largest prime factor of the number 600851475143?

Quick (and cheating) way to factor integers is to use `factorint()` in `sympy.ntheory` module.

In [12]:
import sympy.ntheory
sympy.ntheory.factorint(13195)

{29: 1, 13: 1, 5: 1, 7: 1}

In [13]:
def prob003_sympy(n:int=600851475143) -> int:
    factors = sympy.ntheory.factorint(n)
    return max(factors)

In [14]:
prob003_sympy()

6857

In [15]:
%timeit prob003_sympy()

1000 loops, best of 3: 532 µs per loop


Many nice algorithms for [integer factorization](https://en.wikipedia.org/wiki/Integer_factorization) exist (such as [Pollard's rho algorithm](https://en.wikipedia.org/wiki/Pollard's_rho_algorithm)), but a simple approach is good enough for this problem.


I prepared a generator giving psudo-prime sequence. I don't worry about composite numbers in the sequence because the target number $n$ is not divisible by them.

In [20]:
def psudo_primes():
    """
    Generate numbers n > 1 that is NOT multiple of 2 or 3.
    
    >>> import itertools
    >>> xs = tuple(itertools.takewhile(lambda x: x < 30, psudo_primes()))
    (2, 3, 5, 7, 11, 13, 17, 19, 23, 25, 29)
    """
    yield 2
    yield 3

    x = 5
    while True:
        yield x
        x += 2
        yield x
        x += 6

def prob003(n:int=600851475143) -> int:
    """
    >>> prob003(13195)
    29
    """
    assert n > 1    
    for p in psudo_primes():
        while n % p == 0:
            n //= p
            maxval = p
        if p * p > n:
            break
    if n > 1:
        maxval = n
    return maxval

In [17]:
prob003()

6857

In [18]:
%timeit prob003()

10000 loops, best of 3: 167 µs per loop


## Problem 4
[Largest palindrome product](http://projecteuler.net/problem=4)
> A palindromic number reads the same both ways. The largest palindrome made from the product of two 2-digit numbers is $9009 = 91 \times 99$.
> 
> Find the largest palindrome made from the product of two 3-digit numbers.

$n$-digit number $x$ is equivalent to $10^{n-1} \leq x < 10^n$. Following brute force algorithm is quadratic time complexity. **[FIXME] It should be faster.**

In [42]:
def is_palindrome(number:int) -> bool:
    """
    Check if number is palindrome, the numbers identical to its reversed-direction digits.

    >>> is_palindrome(15651)
    True

    >>> is_palindrome(56)
    False
    """
    s = str(number)
    return s == ''.join(reversed(s))


def prob004(digits:int=3):
    """
    >>> prob004(digits=2)
    (9009, 91, 99)
    """
    lower_limit = 10 ** (digits - 1)
    upper_limit = 10 ** digits
    result = (0, 0, 0)
    for i in range(lower_limit, upper_limit):
        for j in range(i, upper_limit):
            x = i * j
            if is_palindrome(x) and result < (x, i, j):
                result = (x, i, j)

    return result

In [43]:
prob004(digits=3)

(906609, 913, 993)

In [44]:
%timeit prob004()

1 loops, best of 3: 738 ms per loop


## Problem 5
[Smallest multiple](http://projecteuler.net/problem=5)
> 2520 is the smallest number that can be divided by each of the numbers from 1 to 10 without any remainder.

> What is the smallest positive number that is evenly divisible by all of the numbers from 1 to 20?

The problem is to find the loweset common multiplier (LCM). `lcm` is available in `sympy`.

In [35]:
import sympy
import functools

functools.reduce(sympy.lcm, range(1,11))

2520

Greatest common divisor (GCD) is actually in the standard library `fractions`. And GCD and LCM are related by the identity

$$ \mathrm{LCM}(a, b) = \frac{a\, b}{\mathrm{GCD}(a, b)}. $$

In [42]:
import fractions

def lcm(a:int, b:int) -> int:
    """
    Return the lowest common multiplier of a and b
    """
    return a // fractions.gcd(a, b) * b

def prob005(maxval:int=20) -> int:
    return functools.reduce(lcm, range(1, maxval+1))

In [43]:
prob005()

232792560

In [44]:
%timeit prob005()

100000 loops, best of 3: 17.6 µs per loop


## Problem 6
[Sum square difference](http://projecteuler.net/problem=6)

> The sum of the squares of the first ten natural numbers is,

> $$ 1^2 + 2^2 + \cdots + 10^2 = 385 $$

> The square of the sum of the first ten natural numbers is,

> $$(1 + 2 + \cdots + 10)^2 = 55^2 = 3025$$

> Hence the difference between the sum of the squares of the first ten natural numbers and the square of the sum is $3025 - 385 = 2640$.

> Find the difference between the sum of the squares of the first one hundred natural numbers and the square of the sum.

Knowing the sums,

\begin{align}
    \sum_{n=1}^{N} n   &= \frac{1}{2} N(N+1), \\
    \sum_{n=1}^{N} n^2 &= \frac{1}{6} N (N+1) (2N + 1).
\end{align}

we can calculate the difference between the sum of squares and the square of the sum.

$$ \left( \sum_{n=1}^{N} n \right)^2 - \sum_{n=1}^{N} n^2  = \frac{1}{12} n (3 n^3 + 2 n^2 - 3n -2). $$



`sympy` can reproduce the algebraic manipulations.

In [50]:
import sympy
# sympy.init_printing()    # turn on sympy printing on Jupyter notebooks

def the_difference():
    i, n = sympy.symbols('i n', integer=True)
    simple_sum = sympy.summation(i, (i, 1, n))
    sum_of_squares = sympy.summation(i**2, (i, 1, n))
    formula = sympy.simplify(simple_sum**2 - sum_of_squares)
    return formula    

def prob006_sympy(nval=100):
    """
    >>> prob006_sympy(10)
    2640
    """
    formula = the_difference()
    return int(formula.evalf(subs={n:nval}))

In [51]:
the_difference()

n*(3*n**3 + 2*n**2 - 3*n - 2)/12

This problem only requires summation over first one hundred numbers so bruteforce approach works as well.

In [53]:
def prob006(n:int=100) -> int:
    """
    >>> prob006(10)
    2640
    """
    xs = range(1, n+1)
    simple_sum = sum(xs)
    sum_of_squares = sum(x**2 for x in xs)
    return simple_sum**2 - sum_of_squares

Then all you need is to evaluate the formula with upper bound.

In [54]:
prob006()

25164150

In [55]:
%timeit prob006()

10000 loops, best of 3: 52.3 µs per loop


## Problem 7

[10001st prime](http://projecteuler.net/problem=7)
> By listing the first six prime numbers: 2, 3, 5, 7, 11, and 13, we can see that the 6th prime is 13.
    
> What is the 10001st prime number?

**Solution:** `sympy.ntheory` package has prime number generators.

In [63]:
sympy.ntheory.prime(6)

13

In [64]:
sympy.ntheory.prime(10001)

104743

s a good enough algorithm to generate prime numbers. 


It is known [(wikipedia.org)](https://en.wikipedia.org/wiki/Prime_number_theorem#Approximations_for_the_nth_prime_number) that  $n$-th prime number $p_n$ is bounded by

$$ \log n + \log \log n - 1 < \frac{p_n}{n} < \log n + \log \log n  \ \ \ \textrm{ for $n \geq 6$}. $$

So, I'll select integers up to the upper bound with [Sieve of Eratosthenes](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes).

In [24]:
import math

# Reproduction from Problem #3
def psudo_primes():
    yield 2
    yield 3
    x = 5
    while True:
        yield x
        x += 2
        yield x
        x += 6


def sieve(n:int) -> [int]:
    """
    Return all prime numbers below n
    
    >>> sieve(10)
    [2, 3, 5, 7]
    """
    assert n > 1
    remaining = [True] * n  # never use the first two elements (0th and 1st)
    for p in range(2, int(math.sqrt(n)+1)):
        if not remaining[p]:
            continue
        for q in range(p*p, n, p):
            remaining[q] = False
    return [p for p in range(2, n) if remaining[p]]


def prime(n:int) -> [int]:
    """
    Return first n prime numbers
    
    >>> prime(10)
    [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
    """
    log = math.log
    if n >= 6:
        upperbound = int(n * (log(n) + log(log(n))))
        out = sieve(upperbound)
    else:
        out = [2, 3, 5, 7, 11]
    return out[:n]    


def prob007(n:int=10001) -> int:
    """
    >>> prob007(6)
    13
    """
    return prime(n)[-1]

In [22]:
prob007()

104743

In [23]:
import sympy
%timeit sympy.ntheory.prime(10001)

The slowest run took 5.39 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 3.15 µs per loop


In [4]:
%timeit prob007()

10 loops, best of 3: 30.3 ms per loop


## Problem 8

[Largest product in a series](http://projecteuler.net/problem=8)
> Find the greatest product of five consecutive digits in the 1000-digit number.

> ```
73167176531330624919225119674426574742355349194934
96983520312774506326239578318016984801869478851843
85861560789112949495459501737958331952853208805511
12540698747158523863050715693290963295227443043557
66896648950445244523161731856403098711121722383113
62229893423380308135336276614282806444486645238749
30358907296290491560440772390713810515859307960866
70172427121883998797908792274921901699720888093776
65727333001053367881220235421809751254540594752243
52584907711670556013604839586446706324415722155397
53697817977846174064955149290862569321978468622482
83972241375657056057490261407972968652414535100474
82166370484403199890008895243450658541227588666881
16427171479924442928230863465674813919123162824586
17866458359124566529476545682848912883142607690042
24219022671055626321111109370544217506941658960408
07198403850962455444362981230987879927244284909188
84580156166097919133875499200524063689912560717606
05886116467109405077541002256983155200055935729725
71636269561882670428252483600823257530420752963450
```

In [85]:
import operator
import functools

def product(iterable):
    return functools.reduce(operator.mul, iterable)

def prob008(s, N=5):
    def slice_N_digits(idx):
        return (int(i) for i in s[idx:idx+N])

    return max(product(slice_N_digits(idx)) for idx in range(len(s)-N))

In [240]:
s = """
73167176531330624919225119674426574742355349194934
96983520312774506326239578318016984801869478851843
85861560789112949495459501737958331952853208805511
12540698747158523863050715693290963295227443043557
66896648950445244523161731856403098711121722383113
62229893423380308135336276614282806444486645238749
30358907296290491560440772390713810515859307960866
70172427121883998797908792274921901699720888093776
65727333001053367881220235421809751254540594752243
52584907711670556013604839586446706324415722155397
53697817977846174064955149290862569321978468622482
83972241375657056057490261407972968652414535100474
82166370484403199890008895243450658541227588666881
16427171479924442928230863465674813919123162824586
17866458359124566529476545682848912883142607690042
24219022671055626321111109370544217506941658960408
07198403850962455444362981230987879927244284909188
84580156166097919133875499200524063689912560717606
05886116467109405077541002256983155200055935729725
71636269561882670428252483600823257530420752963450"""

xs = [int(c) for c in s.replace("\n", '')]
prob008(xs)

In [238]:
%timeit prob008(xs)

100 loops, best of 3: 6.48 ms per loop


## Problem 9
[Special Pythagorean triplet](http://projecteuler.net/problem=9)
> A Pythagorean triplet is a set of three natural numbers, $a < b < c$, for which,

> $$ a^2 + b^2 = c^2 $$

> For example, $3^2 + 4^2 = 9 + 16 = 25 = 5^2.$

> There exists exactly one Pythagorean triplet for which $a + b + c = 1000$.

> Find the product $abc$.

In [17]:
def prob009(total=1000) -> (int, int, int, int):
    """
    >>> prob009(total=12)
    (3, 4, 5, 60)
    """
    N = total // 2   # just an upper bound
    for a in range(1, N):
        for b in range(a, N):
            c = total - a - b
            if a**2 + b**2 == c**2:
                return (a, b, c, a*b*c)
    else:
        return None

In [18]:
prob009(12)

(3, 4, 5, 60)

In [19]:
%timeit prob009()

10 loops, best of 3: 126 ms per loop


## Problem 10
[Summation of primes](http://projecteuler.net/problem=10)
> The sum of the primes below 10 is $2 + 3 + 5 + 7 = 17.$

> Find the sum of all the primes below two million.

`sympy` module has `primerange`.

In [33]:
list(sympy.sieve.primerange(1, 10))

[2, 3, 5, 7]

So, Just take sum of the prime numbers between 1 and 200000.

In [40]:
sympy.sieve??

In [34]:
import sympy

def prob010_sympy(n=2000000):
    return sum(sympy.sieve.primerange(1, n))

In [38]:
%timeit prob010_sympy()

10 loops, best of 3: 63.3 ms per loop


Use Eratosthenes sieve in Problem 7.

In [25]:
# Reproduction from problem 7
def sieve(n:int) -> [int]:
    assert n > 1
    remaining = [True] * (n + 1)  # never use the first two elements (0th and 1st)
    for p in range(2, int(math.sqrt(n) + 1)):
        if not remaining[p]:
            continue
        for q in range(p*p, n+1, p):
            remaining[q] = False
    return [p for p in range(2, n+1) if remaining[p]]


def prob010(n=2000000):
    """
    >>> prob010(10)
    17
    """
    return sum(sieve(n))

In [56]:
prob010()

142913828922

In [57]:
%timeit prob010()

1 loops, best of 3: 717 ms per loop
